In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import os
import wandb

wandb.login(key="8a88a8c49d1c2d31b8677fe0b8eb7d3e3a031f83")
api = wandb.Api()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /export/home/0schindl/.netrc
wandb: Currently logged in as: belaschindler (belaschindler-university-hamburg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [17]:
def get_sweep_data(runs, attributes_config=[], attributes_summary=[], include_run_id=True):
    sweep_data = []
    for run in runs:
        config = run.config
        summary = run.summary

        if summary.get("task_mean/acc") is not None and run.state == "finished":
            run_data = dict()
            if include_run_id:
                run_data["run_id"] = run.id
            # Add the config attributes to the run_data dictionary
            for attr in attributes_config:
                run_data[attr] = config.get(attr)

            # Add the summary attributes to the run_data dictionary
            for attr in attributes_summary:
                run_data[attr] = summary.get(attr)

            sweep_data.append(run_data)

    return sweep_data

In [25]:
_42_adapter_performance = [
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/6kim8tiu", # DIL
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/jdpa9z1x", # Cars
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/cjddpel4", # Imagenet-a
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/hxigp6ck", # Imagenet-r
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/p7zmthx9", # CIL
    ]
datsets_in_CIL = ["cifar100", "cub", "vtab", "omnibenchmark"]


table_421 = []
for i, s in enumerate(_42_adapter_performance):
    sweep = api.sweep(s)
    runs = sweep.runs

    attributes_config = ["dataset", "selection_method"]
    attributes_summary = ["task_mean/acc"]

    data = get_sweep_data(runs, attributes_config, attributes_summary)
    for e in data:
        if i == 4:
            # CIL
            if e["dataset"] in datsets_in_CIL:
                table_421.append(e)
        else:
            table_421.append(e)



df_sweep = pd.DataFrame(table_421)
#print("\nDataFrame der Sweep-Runs:")
#print(df_sweep)
average_accuracy_per_dataset = df_sweep.groupby('dataset')['task_mean/acc'].mean().round(3) * 100
print(average_accuracy_per_dataset)


dataset
cars                 34.4
cddb                 59.6
cifar100             77.1
cub                  72.5
dil_imagenetr        68.8
imageneta            39.5
imagenetr            66.3
limited_domainnet    45.0
omnibenchmark        58.6
vtab                 78.2
Name: task_mean/acc, dtype: float64


In [36]:
_43_selection_method = [
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/wk4w5q0t", # CIL
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/6uix7luf", # CIL cosine
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/lpeje9a7"  # DIL
    ]
datsets_in_CIL = ["cifar100", "cub", "vtab", "omnibenchmark", "imageneta"]

table_431 = []
for i , s in enumerate(_43_selection_method):
    sweep = api.sweep(s)
    runs = sweep.runs

    attributes_config = ["dataset", "selection_method"]
    attributes_summary = ["task_mean/acc"]

    data = get_sweep_data(runs, attributes_config, attributes_summary, include_run_id=False)
    for e in data:
        if i == 0:
            if e["dataset"] in datsets_in_CIL:
                table_431.append(e)
        else:
            table_431.append(e)

df_sweep = pd.DataFrame(table_431)
df_sweep["task_mean/acc"] = df_sweep["task_mean/acc"].astype(float).round(3) * 100
pd.set_option('display.max_rows', None)
print(df_sweep)


              dataset selection_method  task_mean/acc
0                vtab       inv_ws_div           78.4
1                vtab           ws_div           78.4
2                vtab       inv_kl_div           78.4
3                vtab           kl_div           78.4
4                vtab   inv_eucld_dist           78.4
5                vtab       eucld_dist           78.4
6                vtab           around           78.4
7            cifar100       inv_ws_div           78.1
8            cifar100           ws_div           77.0
9            cifar100       inv_kl_div           77.3
10           cifar100           kl_div           77.0
11           cifar100   inv_eucld_dist           77.4
12           cifar100       eucld_dist           77.0
13           cifar100           around           77.2
14      omnibenchmark       inv_ws_div           59.0
15      omnibenchmark   inv_eucld_dist           56.4
16      omnibenchmark           ws_div           58.8
17      omnibenchmark       

In [37]:
pivot_df = df_sweep.pivot(index='selection_method', columns='dataset', values='task_mean/acc')

# Reset des Index, um 'selection_method' als normale Spalte zu erhalten
pivot_df = pivot_df.reset_index()

# Anzeigen des transformierten DataFrames
print(pivot_df)


dataset selection_method  cars  cddb  cifar100   cub  dil_imagenetr  imageneta  imagenetr  limited_domainnet  omnibenchmark  vtab
0                 around  33.5  61.7      77.2  72.2           66.8       39.7       65.9               41.7           58.8  78.4
1             eucld_dist  35.7  61.7      77.0  72.6           66.5       39.4       65.8               42.8           58.2  78.4
2         inv_eucld_dist  38.3  61.7      77.4  72.5           69.3       40.1       67.6               42.6           56.4  78.4
3             inv_kl_div  34.7  61.7      77.3  72.7           68.4       40.3       68.0               44.0           55.8  78.4
4             inv_ws_div  36.0  61.7      78.1  73.1           69.8       40.0       67.0               44.7           59.0  78.4
5                 kl_div  38.3  61.7      77.0  72.6           66.0       40.2       66.2               42.8           56.0  78.4
6                 ws_div  36.2  61.7      77.0  72.7           66.1       39.1       66.6 

In [47]:
def create_colored_table(df, filename):
    output_folder = "local_saved_graphics"
    if not filename.endswith(".png"):
        filename = filename + ".png"
    filepath = os.path.join(output_folder, filename)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Identify numeric columns
    numeric_df = df.select_dtypes(include='number')
    non_numeric_df = df.drop(columns=numeric_df.columns)

    # Create color-coded cells
    cellColours = []
    for i in range(df.shape[0]):
        row_colors = []
        for j in range(df.shape[1]):
            if df.columns[j] in numeric_df.columns:
                col = numeric_df[df.columns[j]]
                value = df.iloc[i, j]
                # Avoid divide-by-zero if min == max
                if col.max() == col.min():
                    normalized_value = 0.5
                else:
                    normalized_value = (value - col.min()) / (col.max() - col.min())
                row_colors.append(plt.cm.Blues(normalized_value))
            else:
                row_colors.append("white")  # No color for non-numeric
        cellColours.append(row_colors)

    # Plot table
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('off')
    table = ax.table(
        cellText=df.values.round(2) if hasattr(df.values[0,0], '__float__') else df.values,
        colLabels=df.columns,
        loc='center',
        cellColours=cellColours
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    plt.savefig(filepath, bbox_inches='tight')
    plt.close(fig)

new_column_names = {
    'cars': 'CARS',
    'cddb': 'CDDB-H',
    'cifar100': 'CIFAR-100',
    'cub': 'CUB',
    'dil_imagenetr': 'IN-R (D)',
    'imageneta': 'IN-A',
    'imagenetr': 'IN-R',
    'limited_domainnet': 'S-DomainNet',
    'omnibenchmark': 'OB',
    'vtab': 'VTAB'
}

# Umbenennen der Spalten
renamed_df = pivot_df.rename(columns=new_column_names)
# Spalten filtern
renamed_df = renamed_df.drop(columns=["VTAB", "CDDB-H"]).round(3)
# --- Erstellen der farbcodierten Tabelle ---
create_colored_table(renamed_df, "selection_method")

In [48]:
table_432 = []
for i , s in enumerate(_43_selection_method):
    sweep = api.sweep(s)
    runs = sweep.runs

    attributes_config = ["dataset", "selection_method"]
    attributes_summary = ["task_mean/acc"]

    data = get_sweep_data(runs, attributes_config, attributes_summary, include_run_id=False)
    for e in data:
        if i == 0:
            if e["dataset"] in datsets_in_CIL:
                table_431.append(e)
        else:
            table_431.append(e)

df_sweep = pd.DataFrame(table_431)
df_sweep["task_mean/acc"] = df_sweep["task_mean/acc"].astype(float).round(3) * 100
pd.set_option('display.max_rows', None)
print(df_sweep)

               dataset selection_method  task_mean/acc
0                 vtab       inv_ws_div           78.4
1                 vtab           ws_div           78.4
2                 vtab       inv_kl_div           78.4
3                 vtab           kl_div           78.4
4                 vtab   inv_eucld_dist           78.4
5                 vtab       eucld_dist           78.4
6                 vtab           around           78.4
7             cifar100       inv_ws_div           78.1
8             cifar100           ws_div           77.0
9             cifar100       inv_kl_div           77.3
10            cifar100           kl_div           77.0
11            cifar100   inv_eucld_dist           77.4
12            cifar100       eucld_dist           77.0
13            cifar100           around           77.2
14       omnibenchmark       inv_ws_div           59.0
15       omnibenchmark   inv_eucld_dist           56.4
16       omnibenchmark           ws_div           58.8
17       o

In [51]:
import numpy as np

def gini_coefficient_formula(task_counts):
    array = np.sort(np.array(task_counts))
    n = array.shape[0]
    index = np.arange(1, n + 1)
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

# Example usage:
task_distribution = [10, 5, 8, 2, 15]
gini = gini_coefficient_formula(task_distribution)
print(f"Gini coefficient for {task_distribution}: {gini:.4f}")

Gini coefficient for [10, 5, 8, 2, 15]: 0.3100
